In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.title("⚡ 5MW Electrolyzer Hardware Criticality Analysis")

costWeight = st.slider("Cost Weight", 0.0, 1.0, 0.45, 0.05)
reliabilityWeight = 1 - costWeight
st.write(f"Reliability Weight: **{reliabilityWeight:.2f}**")

hardwareData = [
    ["MV/LV Transformer", 450000, 120000, 9.5, 9.0],
    ["Converter Stack (IGBTs)", 380000, 180000, 9.0, 8.5],
    ["MV Switchgear", 280000, 85000, 7.5, 8.0],
    ["Liquid Cooling System", 220000, 160000, 6.5, 7.5],
    ["DC Bus Capacitors", 95000, 140000, 6.0, 7.0],
    ["Gate Drivers & Protection", 85000, 60000, 5.5, 8.5],
    ["Harmonic Filters", 180000, 95000, 6.5, 6.0],
    ["MV Protection Relays", 65000, 35000, 4.5, 8.0],
    ["DC Chokes/Reactors", 75000, 25000, 5.0, 6.5],
    ["ATEX Equipment (H2 Zone)", 120000, 80000, 6.0, 7.5],
    ["Auxiliary Power (UPS/DC-DC)", 55000, 45000, 4.0, 7.0],
    ["Transformer Cooling (ONAF)", 45000, 65000, 4.5, 6.5],
]

df = pd.DataFrame(hardwareData, columns=["Component", "CAPEX", "OPEX15yr", "CostScore", "ReliabilityScore"])

df["TotalCost"] = df["CAPEX"] + df["OPEX15yr"]
df["OPEX_Ratio"] = df["OPEX15yr"] / df["TotalCost"]

df["CostImpact"] = (df["CostScore"] * 0.6) + (df["OPEX_Ratio"] * 10 * 0.4)
df["CI"] = (costWeight * df["CostImpact"]) + (reliabilityWeight * df["ReliabilityScore"])
df = df.sort_values(by="CI", ascending=False)
df["Rank"] = range(1, len(df) + 1)

st.subheader("📊 Criticality Ranking Table")
st.dataframe(df[['Rank','Component','CI','CAPEX','OPEX15yr','CostImpact','ReliabilityScore']])

st.subheader("🔥 Criticality Index Bar Chart")
fig_bar = px.bar(df, x="CI", y="Component", orientation="h", color="CI", color_continuous_scale="RdYlGn_r")
st.plotly_chart(fig_bar, use_container_width=True)

st.subheader("⚖️ Cost vs Reliability Matrix")
fig_scatter = px.scatter(df, x="CostImpact", y="ReliabilityScore", size="CI", color="Rank", text="Component",
                         color_continuous_scale="RdYlGn_r")
fig_scatter.update_traces(textposition="top center")
st.plotly_chart(fig_scatter, use_container_width=True)
